# Exercício de Fundamentos e Práticas de Deep Learning - Marco Aurélio Garcia
## Criação de Uma Base de Dados e Treinamento da Rede YOLO

O presente notebook tem o objetivo de resolver o desafio final do curso de Machine Learning - Fundamentos e Práticas de Deep Learning, criando uma nova base de dados para treinamento da rede YOLO e realizando o treinamento com base nos pesos aferidos em treinamento pregresso através de transfer learning; por fim é realizada uma avaliação em uma série de imagens não rotuladas.

Para atender o desafio foram coletadas para treinamento 10 imagens de cachorros da raça Akita e outras 10 imagens de cachorros da raça Shiba, ambos de origem Japonesa. Adicionalmente foram coletadas outras 15 imagens para validação manual dos resultados.
As 20 imagens foram segmentadas e classificadas, gerando as anotações em formato COCO que necessitaram conversão para formato YOLO.

O treinamento da rede foi realizado gerando um novo arquivo de pesos que foi por fim utilizado para testar a identificação dos cães e classificação das raças nas 15 imagens de validação.



Para fazer uso da rede YOLO, utilizamos a implementação de Aleksei Bochkovskii, presente em https://github.com/AlexeyAB/darknet por facilitar a instalação em ambiente windows. Adicionalmente achamos uma ótima orientação passo a passo presente blog https://graiphic.io/setup-darknet-environment/, escrita por Corentin Maravat.

Para criação da base de dados para treinamento, utilizamos VGG Image Anotator (VIA) https://www.robots.ox.ac.uk/~vgg/software/via/. Criado pelo Visual Geometry Group da Universidade de Oxford, trata-se de uma ferramenta standalone simples para anotação em imagens, áudios e vídeos. Para criar as anotações conforme formato exigido pela rede YOLO, foi necessário também criar um notebook para converter as anotações geradas pelo VIA, neste caso achamos muito útil a orientação de conversão de formato COCO para YOLO, documentada por Abdul Rehman em https://medium.com/red-buffer/converting-a-custom-dataset-from-coco-format-to-yolo-format-6d98a4fd43fc.

Abaixo listo outras referências utilizadas como pesquisa para este trabalho:





## Conversão das anotações em formato COCO para YOLO



In [1]:
# Importação dos pacotes necessários para o processo
import json
import cv2
import os
import matplotlib.pyplot as plt
import shutil

In [2]:
# Definição dos diretórios onde estão as imagens originais e destino onde serão depositados os novos arquivos e suas anotações convertidas para YOLO

# Diretório do inu_dataset
input_path = "../Downloads/yolo_inu_classification/Inu_dataset/"

# O diretório de destino deve ser configurado com o caminho onde está instalado a rede darkent YOLO. Deve ser criado o diretório "inu" em data.
output_path = "../darknet-master/build/darknet/x64/data/inu/"
# diretório data utilizado pela rede YOLO
data_path = "../darknet-master/build/darknet/x64/data/"
# diretório onde está o arquivo Darknet.exe
darknet_path = "../darknet-master/"

In [3]:
# Realiza a leitura do arquivo de anotações em formato COCO e deposita seu conteúdo na variável "coco_annotation"
f = open(input_path + 'New_Inu_Dataset___coco.json')
coco_annotation = json.load(f)
f.close()


In [5]:
# Define uma lista com o nome original de cada arquivo de imagem, adicionalmente define uma função para obter os nomes dos arquivos, 
# copiando-os com novos nomes para o diretório de destino

file_names = []

def load_images_from_folder(folder):
  count = 0
  for filename in os.listdir(folder):
        tipo_arquivo = os.path.splitext(filename)[1]
        if os.path.splitext(filename)[1].lower() in ['.jpg','.png','.jpeg']:
            source = os.path.join(folder,filename)
            destination = f"{output_path}/img{count}.jpg"
            try:
                shutil.copy(source, destination)
                # Caso o arquivo já exista
            except shutil.SameFileError:
                print("Já existe um arquivo com este mesmo nome no destino: ", filename)

            file_names.append(filename)
            count += 1

load_images_from_folder(input_path + '/original_dataset/')
print("Lista de arquivos lidos: ", file_names)

Lista de arquivos lidos:  ['ak1.jpg', 'ak10.jpg', 'ak2.jpg', 'ak3.jpeg', 'ak4.jpeg', 'ak5.jpg', 'ak6.jpg', 'ak7.jpg', 'ak8.jpg', 'ak9.jpg', 'sh1.jpg', 'sh10.jpg', 'sh2.jpg', 'sh3.jpg', 'sh4.jpg', 'sh5.jpg', 'sh6.jpg', 'sh7.jpg', 'sh8.jpg', 'sh9.jpg']


In [6]:
# Define uma função para capturar, nas anotações em COCO, o objeto de anotação de uma dada image atraves de seu ID que é passado como parâmetro

def get_img_ann(image_id):
    img_ann = []
    isFound = False
    for ann in coco_annotation['annotations']:
        if ann['image_id'] == image_id:
            img_ann.append(ann)
            isFound = True
    if isFound:
        return img_ann
    else:
        return None

In [7]:
# Define uma função para capturar, nas anotações em COCO, as informações sobre a imagem cujo nome (path completo) foi passado como parâmetro
def get_img(filename):
  for img in coco_annotation['images']:
    if img['file_name'] == filename:
      return img

In [8]:
# Para cada nome de arquivo presente na lista file_names é realizada:
# Extração de informações relevantes sobre a imagem nas anotações COCO
# Coleta das anotações referentes à imagem
# Criação do arquivo individual de anotação da imagem em formato YOLO
# Coleta das coordenadas do retângulo que segmenta o objeto na imagem através do objeto json bbox (bounding box - caixa delimitadora)
# Converte a categoria do objeto o ID da classe - 1 e realiza a conversão das coordenadas 

# A Etiqueta YOLO para cada objeto na imagem tem o seguinte formato: <classe> <centro_x> <centro_y> <largura> <altura>, 
# Ex: 0 0.476837 0.565947 0.384984 0.671463
# Cada objeto presente na imagem deve ter uma linha, onde:
# <class>: O rótulo de classe do objeto.
# <x_center>: A coordenada x normalizada do centro da caixa delimitadora.
# <y_center>: A coordenada y normalizada do centro da caixa delimitadora.
# <largura>: A largura normalizada da caixa delimitadora.
# <altura>: A altura normalizada da caixa delimitadora.


count = 0

for filename in file_names:
  # Extração de informações relevantes sobre a imagem nas anotações COCO
  img = get_img(filename)
  # print(filename)
  img_id = img['id']
  img_w = img['width']
  img_h = img['height']

  # Coleta das anotações referentes à imagem
  img_ann = get_img_ann(img_id)

  if img_ann:
    # Criação do arquivo individual de anotação da imagem em formato YOLO
    file_object = open(f"{output_path}/img{count}.txt", "a")

    for ann in img_ann:
      
      # Coleta das coordenadas do retângulo que segmenta o objeto na imagem através do objeto json bbox (bounding box - caixa delimitadora)
      current_bbox = ann['bbox']
      x = current_bbox[0]
      y = current_bbox[1]
      w = current_bbox[2]
      h = current_bbox[3]

      # Converte a categoria do objeto o ID da classe - 1 e realiza a conversão das coordenadas 
      current_category = ann['category_id'] - 1 # As yolo format labels start from 0 
      # Calcula os pontos centrais do bounding box
      x_centre = (x + (x+w))/2
      y_centre = (y + (y+h))/2
      
      # Normaliza os valores 
      x_centre = x_centre / img_w
      y_centre = y_centre / img_h
      w = w / img_w
      h = h / img_h
      
      # formata os valores com seis casas decimais
      x_centre = format(x_centre, '.6f')
      y_centre = format(y_centre, '.6f')
      w = format(w, '.6f')
      h = format(h, '.6f')
          
      # Escreve a linha com a anotação referente ao objeto na etiqueta da imagem
      file_object.write(f"{current_category} {x_centre} {y_centre} {w} {h}\n")
      print(filename, ": ", f"{current_category} {x_centre} {y_centre} {w} {h}")

    file_object.close()
    count += 1  # This should be outside the if img_ann block.

ak1.jpg :  0 0.476837 0.565947 0.384984 0.671463
ak10.jpg :  0 0.651699 0.558528 0.633495 0.862876
ak10.jpg :  0 0.177184 0.436455 0.344660 0.852843
ak2.jpg :  0 0.607812 0.529167 0.526563 0.922222
ak3.jpeg :  0 0.582103 0.471510 0.830582 0.923077
ak4.jpeg :  0 0.577344 0.470641 0.438281 0.761534
ak5.jpg :  0 0.461111 0.403125 0.901852 0.795139
ak6.jpg :  0 0.189941 0.359010 0.256836 0.547455
ak6.jpg :  0 0.433105 0.614168 0.213867 0.713893
ak6.jpg :  0 0.617188 0.596286 0.169922 0.702889
ak6.jpg :  0 0.829102 0.595598 0.238281 0.726272
ak7.jpg :  0 0.357823 0.565737 0.650340 0.796813
ak8.jpg :  0 0.486667 0.488839 0.937778 0.941964
ak9.jpg :  0 0.471042 0.574742 0.895753 0.840206
sh1.jpg :  1 0.620500 0.541979 0.745000 0.877061
sh10.jpg :  1 0.320261 0.421546 0.352941 0.576112
sh2.jpg :  1 0.549618 0.515625 0.526718 0.864583
sh3.jpg :  1 0.432616 0.553230 0.391547 0.836124
sh4.jpg :  1 0.609091 0.502778 0.350649 0.831481
sh4.jpg :  1 0.269481 0.491667 0.300000 0.831481
sh5.jpg :  1 0.

In [9]:
# Transfere os arquivos de configuração para treinamento da rede YOLO para os diretórios adequados na estrutura darknet
try:
    # Arquivo com as informações sobre o dataset
    shutil.copy(os.path.join(input_path,"inu.data"), os.path.join(data_path,"inu.data")) 
    # Arquivo com o nome das classes na ordem equivalente aos seus IDs COCO
    shutil.copy(os.path.join(input_path,"inu.names"), os.path.join(data_path,"inu.names")) 
    # Relação de arquivos para testes 
    shutil.copy(os.path.join(input_path,"inu_test.txt"), os.path.join(data_path,"inu_test.txt")) 
    # Relação de arquivos para treinamento
    shutil.copy(os.path.join(input_path,"inu_train.txt"), os.path.join(data_path,"inu_train.txt"))
    # Configurações da rede YOLO para treinamento com as classes e quantidade de arquivos deste dataset
    shutil.copy(os.path.join(input_path,"yolo-inu.cfg"), os.path.join(darknet_path,"yolo-inu.cfg"))
    # Pesos aferidos durante o treinamento, após 2000 épocas.
    shutil.copy(os.path.join(input_path,"yolo-inu_2000.weights"), os.path.join(darknet_path,"yolo-inu_2000.weights"))  
except shutil.SameFileError as e:
    print(f"Falha na cópia dos arquivos: {e}")
